In [1]:
!which pip

/home/ale/.local/bin/pip


In [2]:
!which python

/home/ale/prg/py/dezoomcamp/venv/bin/python


In [3]:
# %pip install -U pandas==1.5.3
import pandas as pd
pd.__version__

'1.5.3'

In [4]:
import pyspark


In [5]:
pyspark.__version__

'3.3.2'

In [6]:
pyspark.__file__

'/home/ale/prg/py/dezoomcamp/hw5/spark/spark-3.3.2-bin-hadoop3/python/pyspark/__init__.py'

In [7]:
from pyspark.sql import SparkSession

In [8]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/04 18:11:19 WARN Utils: Your hostname, MSI resolves to a loopback address: 127.0.1.1; using 172.17.101.240 instead (on interface eth0)
24/03/04 18:11:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/04 18:11:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [51]:
df_zone = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv')

In [52]:
df_zone.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [9]:
df.write.parquet('zones')

In [12]:
! ls -lah

total 19M
drwxr-xr-x  6 ale ale 4.0K Mar  4 17:37 .
drwxr-xr-x 10 ale ale 4.0K Mar  4 15:36 ..
drwxr-xr-x  2 ale ale 4.0K Mar  4 17:28 .ipynb_checkpoints
-rw-r--r--  1 ale ale  389 Mar  4 16:37 exports
-rw-r--r--  1 ale ale  19M Mar  4 17:32 fhv_tripdata_2019-10.csv.gz
drwxr-xr-x  2 ale ale 4.0K Mar  4 17:01 notebooks
-rw-r--r--  1 ale ale 6.7K Mar  4 17:37 pyspark.ipynb
drwxr-xr-x  4 ale ale 4.0K Mar  4 16:07 spark
-rw-r--r--  1 ale ale  13K Aug 17  2016 taxi+_zone_lookup.csv
drwxr-xr-x  2 ale ale 4.0K Mar  4 17:04 zones


In [13]:
! wc -l fhv_tripdata_2019-10.csv.gz

62958 fhv_tripdata_2019-10.csv.gz


In [14]:
df = spark.read.option("header", "true").csv('fhv_tripdata_2019-10.csv.gz')

In [18]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', StringType(), True), StructField('DOlocationID', StringType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [32]:
!head -n 101 fhv_tripdata_2019-10.csv > head.csv

In [33]:
!wc -l head.csv

101 head.csv


In [27]:
!head -n 3 head.csv

dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
B00009,2019-10-01 00:23:00,2019-10-01 00:35:00,264,264,,B00009
B00013,2019-10-01 00:11:29,2019-10-01 00:13:22,264,264,,B00013


In [9]:
df_pandas = pd.read_csv('head.csv')

In [10]:
df_pandas.dtypes

dispatching_base_num       object
pickup_datetime            object
dropOff_datetime           object
PUlocationID                int64
DOlocationID                int64
SR_Flag                   float64
Affiliated_base_number     object
dtype: object

In [16]:
from pyspark.sql import types
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropOff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True),
])

In [17]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhv_tripdata_2019-10.csv')

In [18]:
df.head(3)

[Row(dispatching_base_num='B00009', pickup_datetime=datetime.datetime(2019, 10, 1, 0, 23), dropOff_datetime=datetime.datetime(2019, 10, 1, 0, 35), PULocationID=264, DOLocationID=264, SR_Flag=None, Affiliated_base_number='B00009'),
 Row(dispatching_base_num='B00013', pickup_datetime=datetime.datetime(2019, 10, 1, 0, 11, 29), dropOff_datetime=datetime.datetime(2019, 10, 1, 0, 13, 22), PULocationID=264, DOLocationID=264, SR_Flag=None, Affiliated_base_number='B00013'),
 Row(dispatching_base_num='B00014', pickup_datetime=datetime.datetime(2019, 10, 1, 0, 11, 43), dropOff_datetime=datetime.datetime(2019, 10, 1, 0, 37, 20), PULocationID=264, DOLocationID=264, SR_Flag=None, Affiliated_base_number='B00014')]

In [20]:
df = df.repartition(6)

In [21]:
df.write.parquet('fhv/')

In [22]:
df = spark.read.parquet('fhv/')

In [25]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [32]:
import datetime

In [38]:
df1 = df.select('pickup_datetime', 'dropOff_datetime').filter((df['pickup_datetime'] >= datetime.datetime(2019, 10, 15, 0, 0, 0)) & (df['pickup_datetime'] <= datetime.datetime(2019, 10, 15, 23, 59, 59)))

In [42]:
df1.count()

62610

In [43]:
df.count()

1897493

In [45]:
from pyspark.sql import functions as F

In [46]:
df2 = df.withColumn('len', df['dropOff_datetime'] - df['pickup_datetime'])

In [50]:
df2.sort('len').tail(5)

[Row(dispatching_base_num='B02921', pickup_datetime=datetime.datetime(2019, 10, 17, 14, 0), dropOff_datetime=datetime.datetime(2020, 10, 18, 0, 0), PULocationID=None, DOLocationID=None, SR_Flag=None, Affiliated_base_number='B03037', len=datetime.timedelta(days=366, seconds=36000)),
 Row(dispatching_base_num='B00746         ', pickup_datetime=datetime.datetime(2019, 10, 1, 21, 43, 42), dropOff_datetime=datetime.datetime(2027, 10, 1, 21, 45, 23), PULocationID=159, DOLocationID=264, SR_Flag=None, Affiliated_base_number='B00746         ', len=datetime.timedelta(days=2922, seconds=101)),
 Row(dispatching_base_num='B02416', pickup_datetime=datetime.datetime(2019, 10, 31, 23, 46, 33), dropOff_datetime=datetime.datetime(2029, 11, 1, 0, 13), PULocationID=None, DOLocationID=None, SR_Flag=None, Affiliated_base_number='B02416', len=datetime.timedelta(days=3653, seconds=1587)),
 Row(dispatching_base_num='B02832', pickup_datetime=datetime.datetime(2019, 10, 11, 18, 0), dropOff_datetime=datetime.date

In [53]:
df3 = df.join(df_zone, (df.PULocationID) == df_zone.LocationID )

In [55]:
df3.head()

Row(dispatching_base_num='B01196', pickup_datetime=datetime.datetime(2019, 10, 3, 9, 59, 46), dropOff_datetime=datetime.datetime(2019, 10, 3, 10, 4, 48), PULocationID=264, DOLocationID=136, SR_Flag=None, Affiliated_base_number='B01196', LocationID='264', Borough='Unknown', Zone='NV', service_zone='N/A')

In [57]:
df3.groupBy('Zone').count().orderBy('count').show()

+--------------------+-----+
|                Zone|count|
+--------------------+-----+
|         Jamaica Bay|    1|
|Governor's Island...|    2|
| Green-Wood Cemetery|    5|
|       Broad Channel|    8|
|     Highbridge Park|   14|
|        Battery Park|   15|
|Saint Michaels Ce...|   23|
|Breezy Point/Fort...|   25|
|Marine Park/Floyd...|   26|
|        Astoria Park|   29|
|    Inwood Hill Park|   39|
|       Willets Point|   47|
|Forest Park/Highl...|   53|
|  Brooklyn Navy Yard|   57|
|        Crotona Park|   62|
|        Country Club|   77|
|     Freshkills Park|   89|
|       Prospect Park|   98|
|     Columbia Street|  105|
|  South Williamsburg|  110|
+--------------------+-----+
only showing top 20 rows

